In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
import os
from datetime import datetime

DIR_PATH = "/kaggle/input/severstal-febmarmay"

Data = pd.read_csv(os.path.join(DIR_PATH, "SensorsData_FebMarMay.csv"), index_col=0)
Data.index = np.array(
    ([datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in Data.index]),
    dtype='datetime64[ns]')

stops_data = pd.read_csv(os.path.join(DIR_PATH, "Jan_to_Jun_StopLogs.csv"))
stops_data['start_datetime'] = stops_data.start_datetime.astype('datetime64[ns]')
stops_data['end_datetime'] = stops_data.end_datetime.astype('datetime64[ns]')

FileNotFoundError: [Errno 2] File /kaggle/input/severstal-febmarmay/SensorsData_FebMarMay.csv does not exist: '/kaggle/input/severstal-febmarmay/SensorsData_FebMarMay.csv'

In [3]:
from sklearn.impute import SimpleImputer

Data.dropna(how='all', inplace=True)
imputer = SimpleImputer()
Data = pd.DataFrame(imputer.fit_transform(Data), index=Data.index, columns=Data.columns)

# Target

In [4]:
# Берем таргет из журнальных данных

Data['target'] = 0
Data['target_dscr'] = "Норма"

for start, end, dscr in stops_data[np.invert(stops_data.description.isin([
        "Технологические плановые : Межплавочный период",
        ]))]\
        [['start_datetime', 'end_datetime', 'description']].values:
    
    Data['target'][(Data.index >= start) & (Data.index <= end)] = 1
    Data['target_dscr'][(Data.index >= start) & (Data.index <= end)] = dscr

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [5]:
# Добавляем таргет по минимумам вибрации

Data["target"][(Data["Вибрация заднего подшипника двигателя (осевая)"] < 15.0)
             & (Data["Вибрация заднего подшипника двигателя (радиальная)"] < 15.0)] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
from copy import deepcopy

def SetShiftTarget(Data, delta):
    """
    Сдвигает изначальный таргет на delta шагов назад
    Шаг дискретизации по времени : 5 минут
    """
    Data["shift_target"] = deepcopy(Data.target.shift(-delta, fill_value=0).values)
    Data["shift_target"] = Data.shift_target.astype(int)
    return Data

In [7]:
# Смещаем на 180 шагов (здесь минут) = 3 часа

Data = SetShiftTarget(Data, 180)

X = Data.drop(['target', 'target_dscr', 'shift_target'], axis=1)
y = Data['shift_target']

X_to_vis = Data.drop(['target', 'target_dscr', 'shift_target'], axis=1)
y_to_vis = Data['shift_target']

#========================================================================

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Normalize X
mms = MinMaxScaler(feature_range=(-1,1))
imputer = SimpleImputer()

X = pd.DataFrame(mms.fit_transform(imputer.fit_transform(X)),
                 index=X.index,
                 columns=X.columns)

X_to_vis = pd.DataFrame(mms.fit_transform(imputer.fit_transform(X_to_vis)),
                        index=X_to_vis.index,
                        columns=X_to_vis.columns)

# Предобработка
## Генерация фичей
### Попарные корреляции

In [8]:
WINDOW_SIZE = 120

corr_X = X.rolling(window = WINDOW_SIZE).corr()

In [9]:
from itertools import product, combinations
from tqdm.notebook import tqdm

Sensors_Combs = list(combinations(X.columns, r=2))
CorrFeatures = pd.DataFrame()

for sens_1, sens_2 in tqdm(Sensors_Combs):

    # Выбираем из датафрейма, корреляцию за все время пары датчиков `sens_1` и `sens_2`
    corr_series = corr_X[sens_1].loc[:, sens_2]

    # Так как использовалась оконная функция в начале есть пропуски
    # Заполним их случайными величинами из диапазона [MIN, MAX] для данного ряда
    MAX, MIN = corr_series.max(), corr_series.min()
    fill_values = np.random.sample(size=WINDOW_SIZE) * (MAX - MIN) + MIN
    corr_series[:WINDOW_SIZE] = fill_values

    # Все такие новые колонки с попарными корреляциями датчиков
    # Будут начинаться с `Corr_`
    corr_series.name = '_'.join(['Corr', sens_1, sens_2])

    CorrFeatures = pd.concat([CorrFeatures, corr_series], axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1043: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.iloc[indexer] = extract_arr

In [10]:
CorrFeatures.dropna(how="any", inplace=True)
CorrFeatures.replace(np.inf, 1, inplace=True)
CorrFeatures.replace(-np.inf, -1, inplace=True)

mms = MinMaxScaler()

CorrFeatures = pd.DataFrame(mms.fit_transform(CorrFeatures),
                            index=CorrFeatures.index,
                            columns=CorrFeatures.columns)

## Объединение

In [11]:
imputer = SimpleImputer()
X = pd.DataFrame(imputer.fit_transform(pd.concat([X, CorrFeatures], axis=1)),
                 index = X.index,
                 columns = X.columns.to_list() + CorrFeatures.columns.to_list())

## Multiplication features : to CNN

The multiplication method is proposed in this work to create more heterogeneous image-like data representation. These heterogeneous patterns in the image representation can help the creation of different filters by the CNN model, increasing its ability to correctly classify the data as normal or faulty. Given the example $d = \{f_1, f_2, …, f_n \}$, the transformed example $D_m$ is given by a matrix created by the multiplication of the $f_i$ features by the whole example $d$, as presented by Equation 5:

$$\begin{equation*}D_{m}=\begin{bmatrix}{f_{1} \cdot f_{1}} & {f_{1} \cdot f_{2}} & {\cdots} & {f_{1} \cdot f_{n}} \\ {f_{2} \cdot f_{1}} & {f_{2} \cdot f_{2}} & {\ddots} & {f_{2} \cdot f_{n}} \\ {\vdots} & {\vdots} & {\ddots} & {\vdots} \\ {f_{n} \cdot f_{1}} & {f_{n} \cdot f_{2}} & {\cdots} & {f_{n} \cdot f_{n}}\end{bmatrix} \tag{5}\end{equation*}$$

In [12]:
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler

def get_X_for_CNN(X):
    
    X_vals = X.values.reshape(*X.shape, 1)
    mms = MinMaxScaler()
    X_to_CNN = np.array([mms.fit_transform(ex @ ex.T) for ex in tqdm(X_vals)], dtype=float)
    
    return X_to_CNN.reshape(-1, *X_to_CNN.shape[1:], 1)

def batch_generator(X, y, batch_size):
    samples_per_epoch = X.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    
    X_vals = X.values.reshape(*X.shape, 1)
    y_vals = y.values
    mms = MinMaxScaler()

    
    while True:
        X_batch = X_vals[batch_size*counter : batch_size*(counter+1)].astype('float32')
        X_to_CNN_batch = np.array([mms.fit_transform(ex @ ex.T) for ex in X_batch], dtype=float)
        y_batch = np.array(y_vals[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        counter += 1
        yield X_to_CNN_batch.reshape(-1,153,153,1), y_batch

        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization

n_hidden_convs = 3
num_filters = 7
filter_size = 3


# def create_model(n_hidden_convs = n_hidden_convs,
#                  n_hidden_filters = num_filters,
#                  kernel_size = filter_size):

#     model = Sequential()
#     model.add(
#         Conv2D(n_hidden_filters,
#                kernel_size,
#                padding='same',
#                activation='tanh',
#                input_shape=X_to_CNN.shape[1:]))
#     for i in range(n_hidden_convs):
#         model.add(
#             Conv2D(n_hidden_filters,
#                    kernel_size,
#                    padding='same',
#                    activation='tanh'))
#     model.add(Conv2D(1, kernel_size, padding='same', activation='tanh'))
#     model.add(MaxPooling2D(2)),
#     model.add(Flatten())
#     model.add(Dense(2, activation='softmax'))

#     model.compile(
#         'adam',
#         loss='categorical_crossentropy',
#         metrics=['accuracy', 'AUC'],
#     )

#     return model


# model = create_model()

#=============================================================================


def create_model(n_hidden_filters = num_filters,
                 kernel_size = filter_size):
    
    model = Sequential([
        Conv2D(n_hidden_filters, kernel_size, input_shape=(153,153,1)),
        BatchNormalization(),
        Conv2D(n_hidden_filters, kernel_size),
        Flatten(),
        Dense(1, activation='sigmoid'),
    ])

    model.compile(
        'adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC'],
    )

    return model

## Обучение

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42,
                                                    shuffle=False)

In [15]:
del(X)

In [3]:
from tensorflow.keras.utils import to_categorical

model = create_model(n_hidden_filters=10, kernel_size=4)

# history = model.fit(X_train,
#                     y_train,
#                     batch_size=180,
#                     epochs=5,
#                     validation_data=(X_test, y_test),
#                     shuffle=False)

BATCH_SIZE = 60

history = model.fit_generator(batch_generator(X_train, y_train, batch_size=BATCH_SIZE),
                              epochs=10,
                              steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
                              validation_data=batch_generator(X_test, y_test, batch_size=BATCH_SIZE*2),
                              validation_steps=X_test.shape[0] // (BATCH_SIZE*2),
                              workers=4,
                              use_multiprocessing=True)

NameError: name 'batch_generator' is not defined

In [21]:
Preds_train = model.predict_generator(
    batch_generator(X_train, y_train, batch_size=BATCH_SIZE),
    steps=X_train.shape[0] // BATCH_SIZE,
    verbose=1,
)

1361/1361 [==============================] - 49s 36ms/step


In [31]:
import plotly.graph_objects as go
import plotly.express as px

config = dict({'scrollZoom': True})

import pandas as pd
sens_name_1 = X_train.columns[12]
sens_name_2 = X_train.columns[13]

Sens_1 = X_train[sens_name_1].values
Sens_2 = X_train[sens_name_2].values




fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train.index, y=Sens_1, name="Вибрация ЗПД (осев.)"))
fig.add_trace(go.Scatter(x=X_train.index, y=Sens_2, name="Вибрация ЗПД (рад.)"))
fig.add_trace(go.Scatter(x=X_train.index, y=Preds_train.ravel(), name="Уровень опасности"))


#==========================================================================================#

# fig.update_xaxes(rangeslider_visible=True)
fig.show(config=config)
# fig.write_html("March_Stops_Example.html", config=config)